In [1]:
# load libraies

%run py_libraries.py

/Users/4476224/.local/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.6.0 and strictly below 2.9.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.13.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
# loading utility files

from utility.sv_fig import savefig
from utility.make_cm import make_confusion_matrix
from utility.get_g_result import get_gamma_results
from utility.plt_result import plot_results

In [3]:
def savefig(filename, crop = True):
    plt.savefig('{}.pdf'.format(filename))

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_row', None)

# load data
X_train = pd.read_csv('data/norm_X_train_NCa_PCa.csv')
X_test = pd.read_csv('data/norm_X_test_NCa_PCa.csv')
y_train = pd.read_csv('data/y_train_NCa_PCa.csv')
y_test = pd.read_csv('data/y_test_NCa_PCa.csv')


print(X_test.shape)

(25, 37)


In [5]:
# Print the column names as a list
print(X_test.columns.tolist())

['ENA.78', 'IFN.y', 'IL.10', 'IL.6', 'IL.8', 'MCP.1', 'MDC', 'MIP.1a', 'TNF.a', 'C.peptide', 'G.CSF', 'IL.22', 'Insulin', 'Leptin', 'MIP.3a', 'GRO.a', 'HGF', 'MMP.2', 'Adiponectin', 'CRP', 'GDF.15', 'TIMP.1', 'TGF.B2', 'TGF.B1', 'PPAR.y', 'HIF.1a', 'Laminin', 'HbA1c', 'CA19.9', 'Glucose', 'HDL', 'CCK', 'LDL', 'Triglyceride', 'Albumin', 'Lumican', 'ZAG']


In [6]:
Xy_NCa_PCa = pd.concat([X_test, y_test], axis=1)

In [7]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from scipy import stats

col_to_scale = ['ENA.78', 'IFN.y', 'IL.10', 'IL.6', 'IL.8', 'MCP.1', 'MDC', 'MIP.1a', 'TNF.a', 'C.peptide', 'G.CSF', 'IL.22', 'Insulin', 'Leptin', 
                'MIP.3a', 'GRO.a', 'HGF', 'MMP.2', 'Adiponectin', 'CRP', 'GDF.15', 'TIMP.1', 'TGF.B2', 'TGF.B1', 'PPAR.y', 'HIF.1a', 'Laminin', 'HbA1c', 
                'CA19.9', 'Glucose', 'HDL', 'CCK', 'LDL', 'Triglyceride', 'Albumin', 'Lumican', 'ZAG']

def midas_impl(missing, col_to_scale, n_epoch):
    missing_cp = missing.copy() # make copy of missing df
    
    # scaler = MinMaxScaler().fit(missing_cp[col_to_scale]) # scale data
    scaler = StandardScaler().fit(missing_cp[col_to_scale]) # scale data
    missing_sc = scaler.transform(missing_cp[col_to_scale]) # this is an array, convert to df

    missing_df = pd.DataFrame({'ENA.78':missing_sc[:,0],
                               'IFN.y':missing_sc[:,1],
                               'IL.10':missing_sc[:,2],
                               'IL.6':missing_sc[:,3],
                               'IL.8':missing_sc[:,4],
                               'MCP.1':missing_sc[:,5],
                               'MDC':missing_sc[:,6],
                               'MIP.1a':missing_sc[:,7],
                               'TNF.a':missing_sc[:,8],
                               'C.peptide':missing_sc[:,9],
                               'G.CSF':missing_sc[:,10],
                               'IL.22':missing_sc[:,11],
                               'Insulin':missing_sc[:,12],
                               'Leptin':missing_sc[:,13],
                               'MIP.3a':missing_sc[:,14],
                               'GRO.a':missing_sc[:,15],
                               'HGF':missing_sc[:,16],
                               'MMP.2':missing_sc[:,17],
                               'Adiponectin':missing_sc[:,18],
                               'CRP':missing_sc[:,19],
                               'GDF.15':missing_sc[:,20],
                               'TIMP.1':missing_sc[:,21],
                               'TGF.B2':missing_sc[:,22],
                               'TGF.B1':missing_sc[:,23],
                               'PPAR.y':missing_sc[:,24],
                               'HIF.1a':missing_sc[:,25],
                               'Laminin':missing_sc[:,26],
                               'HbA1c':missing_sc[:,27],
                               'CA19.9':missing_sc[:,28],
                               'Glucose':missing_sc[:,29],
                               'HDL':missing_sc[:,30],
                               'CCK':missing_sc[:,31],
                               'LDL':missing_sc[:,32],
                               'Triglyceride':missing_sc[:,33],
                               'Albumin':missing_sc[:,34],
                               'Lumican':missing_sc[:,35],
                               'ZAG':missing_sc[:,36],
                               })
    
    # Extract cat columns from missing
    ext_col1 = missing_cp['CACHEXSTAGE0VIG']
    

    # Add the extracted cat columns to missing_df
    missing_df = pd.concat([missing_df, ext_col1.rename('CACHEXSTAGE0VIG')], axis=1)
    

    missing_df.to_csv('missing_test_NCa_PCa.csv')
    data_0 = pd.read_csv('missing_test_NCa_PCa.csv')
    data_0.columns.str.strip()

    categorical = ['CACHEXSTAGE0VIG']
    
    data_cat, cat_cols_list = md.cat_conv(data_0[categorical])

    data_0.drop(categorical, axis = 1, inplace = True)
    constructor_list = [data_0]
    constructor_list.append(data_cat)
    data_in = pd.concat(constructor_list, axis=1)

    na_loc = data_in.isnull()
    data_in[na_loc] = np.nan

    # imputer = md.Midas(layer_structure = [128,128], vae_layer = True, seed = 89, input_drop = 0.75, learn_rate = 0.0001)
    imputer = md.Midas(layer_structure = [128,128], vae_layer = False, seed = 42, input_drop = 0.75, learn_rate = 0.001)
    imputer.build_model(data_in, softmax_columns = cat_cols_list)
    imputer.train_model(training_epochs = n_epoch)

    # do not update M
    imputations = imputer.generate_samples(m=20).output_list

    flat_cats = [cat for variable in cat_cols_list for cat in variable]
    for i in range(len(imputations)):
        tmp_cat = [imputations[i][x].idxmax(axis=1) for x in cat_cols_list]
        cat_df = pd.DataFrame({categorical[i]:tmp_cat[i] for i in range(len(categorical))})
        imputations[i] = pd.concat([imputations[i], cat_df], axis = 1).drop(flat_cats, axis = 1)

    mapping_ca = {'CACHEXSTAGE0VIG_0.0': 0,'CACHEXSTAGE0VIG_1.0': 1}

    imputations[0] = imputations[0].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[1] = imputations[1].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[2] = imputations[2].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[3] = imputations[3].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[4] = imputations[4].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[5] = imputations[5].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[6] = imputations[6].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[7] = imputations[7].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[8] = imputations[8].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[9] = imputations[9].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[10] = imputations[10].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[11] = imputations[11].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[12] = imputations[12].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[13] = imputations[13].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[14] = imputations[14].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[15] = imputations[15].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[16] = imputations[16].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[17] = imputations[17].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[18] = imputations[18].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[19] = imputations[19].replace({'CACHEXSTAGE0VIG': mapping_ca})

    # rescale imputations
    imputations[0][col_to_scale] = scaler.inverse_transform(imputations[0][col_to_scale])
    imputations[1][col_to_scale] = scaler.inverse_transform(imputations[1][col_to_scale])
    imputations[2][col_to_scale] = scaler.inverse_transform(imputations[2][col_to_scale])
    imputations[3][col_to_scale] = scaler.inverse_transform(imputations[3][col_to_scale])
    imputations[4][col_to_scale] = scaler.inverse_transform(imputations[4][col_to_scale])
    imputations[5][col_to_scale] = scaler.inverse_transform(imputations[5][col_to_scale])
    imputations[6][col_to_scale] = scaler.inverse_transform(imputations[6][col_to_scale])
    imputations[7][col_to_scale] = scaler.inverse_transform(imputations[7][col_to_scale])
    imputations[8][col_to_scale] = scaler.inverse_transform(imputations[8][col_to_scale])
    imputations[9][col_to_scale] = scaler.inverse_transform(imputations[9][col_to_scale])
    imputations[10][col_to_scale] = scaler.inverse_transform(imputations[10][col_to_scale])
    imputations[11][col_to_scale] = scaler.inverse_transform(imputations[11][col_to_scale])
    imputations[12][col_to_scale] = scaler.inverse_transform(imputations[12][col_to_scale])
    imputations[13][col_to_scale] = scaler.inverse_transform(imputations[13][col_to_scale])
    imputations[14][col_to_scale] = scaler.inverse_transform(imputations[14][col_to_scale])
    imputations[15][col_to_scale] = scaler.inverse_transform(imputations[15][col_to_scale])
    imputations[16][col_to_scale] = scaler.inverse_transform(imputations[16][col_to_scale])
    imputations[17][col_to_scale] = scaler.inverse_transform(imputations[17][col_to_scale])
    imputations[18][col_to_scale] = scaler.inverse_transform(imputations[18][col_to_scale])
    imputations[19][col_to_scale] = scaler.inverse_transform(imputations[19][col_to_scale])

    n=0
    for i in imputations:
        file_out_csv = "midas_test_NCa_PCa/midas_NCa_PCa_" + str(n) + ".csv"
        i.to_csv(file_out_csv, index=False)
        n += 1

    df0 = pd.read_csv('midas_test_NCa_PCa/midas_NCa_PCa_0.csv'); df0 = df0.iloc[:,1:]
    df1 = pd.read_csv('midas_test_NCa_PCa/midas_NCa_PCa_1.csv'); df1 = df1.iloc[:,1:]
    df2 = pd.read_csv('midas_test_NCa_PCa/midas_NCa_PCa_2.csv'); df2 = df2.iloc[:,1:]
    df3 = pd.read_csv('midas_test_NCa_PCa/midas_NCa_PCa_3.csv'); df3 = df3.iloc[:,1:]
    df4 = pd.read_csv('midas_test_NCa_PCa/midas_NCa_PCa_4.csv'); df4 = df4.iloc[:,1:]
    df5 = pd.read_csv('midas_test_NCa_PCa/midas_NCa_PCa_5.csv'); df5 = df5.iloc[:,1:]
    df6 = pd.read_csv('midas_test_NCa_PCa/midas_NCa_PCa_6.csv'); df6 = df6.iloc[:,1:]
    df7 = pd.read_csv('midas_test_NCa_PCa/midas_NCa_PCa_7.csv'); df7 = df7.iloc[:,1:]
    df8 = pd.read_csv('midas_test_NCa_PCa/midas_NCa_PCa_8.csv'); df8 = df8.iloc[:,1:]
    df9 = pd.read_csv('midas_test_NCa_PCa/midas_NCa_PCa_9.csv'); df9 = df9.iloc[:,1:]
    df10 = pd.read_csv('midas_test_NCa_PCa/midas_NCa_PCa_10.csv'); df10 = df10.iloc[:,1:]
    df11 = pd.read_csv('midas_test_NCa_PCa/midas_NCa_PCa_11.csv'); df11 = df11.iloc[:,1:]
    df12 = pd.read_csv('midas_test_NCa_PCa/midas_NCa_PCa_12.csv'); df12 = df12.iloc[:,1:]
    df13 = pd.read_csv('midas_test_NCa_PCa/midas_NCa_PCa_13.csv'); df13 = df13.iloc[:,1:]
    df14 = pd.read_csv('midas_test_NCa_PCa/midas_NCa_PCa_14.csv'); df14 = df14.iloc[:,1:]
    df15 = pd.read_csv('midas_test_NCa_PCa/midas_NCa_PCa_15.csv'); df15 = df15.iloc[:,1:]
    df16 = pd.read_csv('midas_test_NCa_PCa/midas_NCa_PCa_16.csv'); df16 = df16.iloc[:,1:]
    df17 = pd.read_csv('midas_test_NCa_PCa/midas_NCa_PCa_17.csv'); df17 = df17.iloc[:,1:]
    df18 = pd.read_csv('midas_test_NCa_PCa/midas_NCa_PCa_18.csv'); df18 = df18.iloc[:,1:]
    df19 = pd.read_csv('midas_test_NCa_PCa/midas_NCa_PCa_19.csv'); df19 = df19.iloc[:,1:]

    # Initialize the output dataframe with the same dimensions
    output_df = pd.DataFrame(index=df0.index, columns=df0.columns)

    # Process continuous columns
    for col in df1.columns[:35]:
        output_df[col] = (abs(df0[col]) + abs(df1[col]) + abs(df2[col]) + abs(df3[col]) + abs(df4[col]) + 
                          abs(df5[col]) + abs(df6[col]) + abs(df7[col]) + abs(df8[col]) + abs(df9[col]) + 
                         abs(df10[col]) + abs(df11[col]) + abs(df12[col]) + abs(df13[col]) + abs(df14[col]) + 
                          abs(df15[col]) + abs(df16[col]) + abs(df17[col]) + abs(df18[col]) + abs(df19[col])) / 20

    # Process categorical columns
    for col in df1.columns[35:]:
        output_df[col] = pd.concat([df0[col],df1[col],df2[col],df3[col],df4[col],df5[col],df6[col],df7[col],df8[col],df9[col],
                                   df10[col],df11[col],df12[col],df13[col],df14[col],df15[col],df16[col],df17[col],df18[col],df19[col]], axis=1).median(axis=1)
        
        # median = stats.median([df0[col],df1[col],df2[col],df3[col],df4[col],df5[col],df6[col],df7[col],df8[col],df9[col],
        #                   df10[col],df11[col],df12[col],df13[col],df14[col],df15[col],df16[col],df17[col],df18[col],df19[col]], axis=0).median[0]
        # output_df[col] = median

    # Write the output dataframe to a CSV file
    output_df.to_csv('output_test_NCa_PCa.csv', index=False)

    # re-order cols in output_df
    output_df_re = output_df[['ENA.78', 'IFN.y', 'IL.10', 'IL.6', 'IL.8', 'MCP.1', 'MDC', 'MIP.1a', 'TNF.a', 'C.peptide', 'G.CSF', 'IL.22', 'Insulin', 
                              'Leptin', 'MIP.3a', 'GRO.a', 'HGF', 'MMP.2', 'Adiponectin', 'CRP', 'GDF.15', 'TIMP.1', 'TGF.B2', 'TGF.B1', 'PPAR.y', 'HIF.1a', 
                              'Laminin', 'HbA1c', 'CA19.9', 'Glucose', 'HDL', 'CCK', 'LDL', 'Triglyceride', 'Albumin', 'Lumican', 'ZAG', 'CACHEXSTAGE0VIG']].copy()

    return output_df_re





In [8]:
# calculate proportion of missingness in ndata_noR

# total num of NaN in the ndata_noR
total_nan_count = X_test.isna().sum().sum()

# total num of cells in ndata_noR
total_cells = X_test.size

# proportion of NaN
NaN_proportion = total_nan_count / total_cells

print('NaN_proportions:',NaN_proportion)


NaN_proportions: 0.021621621621621623


In [9]:
# 1.35% missingness

Xy_NCa_PCa_md = midas_impl(Xy_NCa_PCa, col_to_scale, 200)

Size index: [38, 2]

Computation graph constructed

Metal device set to: Apple M1 Pro
Model initialised

Epoch: 0 , loss: 56.81471633911133
Epoch: 1 , loss: 66.12118530273438
Epoch: 2 , loss: 89.77793884277344
Epoch: 3 , loss: 124.53736877441406
Epoch: 4 , loss: 151.57737731933594
Epoch: 5 , loss: 203.28683471679688
Epoch: 6 , loss: 235.95310974121094
Epoch: 7 , loss: 330.8088684082031
Epoch: 8 , loss: 172.93052673339844
Epoch: 9 , loss: 231.8455810546875
Epoch: 10 , loss: 286.333984375
Epoch: 11 , loss: 142.91600036621094
Epoch: 12 , loss: 178.2801971435547
Epoch: 13 , loss: 186.2247772216797
Epoch: 14 , loss: 216.0680694580078
Epoch: 15 , loss: 101.79944610595703
Epoch: 16 , loss: 178.18832397460938
Epoch: 17 , loss: 181.58193969726562
Epoch: 18 , loss: 72.31937408447266
Epoch: 19 , loss: 95.2193374633789
Epoch: 20 , loss: 76.24188232421875
Epoch: 21 , loss: 101.54454040527344
Epoch: 22 , loss: 122.64280700683594
Epoch: 23 , loss: 125.9254150390625
Epoch: 24 , loss: 65.89933776855469

In [10]:
Xy_NCa_PCa_md

,ENA.78,IFN.y,IL.10,IL.6,IL.8,MCP.1,MDC,MIP.1a,TNF.a,C.peptide,G.CSF,IL.22,Insulin,Leptin,MIP.3a,GRO.a,HGF,MMP.2,Adiponectin,CRP,GDF.15,TIMP.1,TGF.B2,TGF.B1,PPAR.y,HIF.1a,Laminin,HbA1c,CA19.9,Glucose,HDL,CCK,LDL,Triglyceride,Albumin,Lumican,ZAG,CACHEXSTAGE0VIG
0,0.906353,0.564966,0.475083,0.452681,0.834031,0.908480,1.003835,0.525058,0.729382,0.841801,0.643905,0.254465,0.537083,0.742718,0.553010,0.910991,0.668230,0.996000,0.974258,0.920809,0.935556,0.906897,0.772082,0.882289,0.422619,0.845950,0.902724,0.895749,0.637125,0.725937,0.719375,0.828662,0.878446,0.978279,0.994963,0.885516,0.933561,1.0
1,0.875380,0.682680,0.173453,0.404020,0.738583,0.829126,0.887340,0.440556,0.697789,0.781124,0.580478,0.202644,0.416163,0.774106,0.586010,0.720616,0.702226,0.926590,0.942575,0.698319,0.774975,0.845535,0.731682,0.827453,0.103339,0.841416,0.861074,0.816975,0.395168,0.820161,0.878388,0.825860,0.854209,0.672188,0.982921,0.941650,0.938872,0.0
2,0.859491,0.533457,0.022883,0.292660,0.679268,0.868635,0.918437,0.423738,0.600460,0.755277,0.594245,0.430006,0.390927,0.602280,0.500222,0.790526,0.561010,0.797051,0.922795,0.759332,0.812744,0.834163,0.695623,0.906347,0.393240,0.442763,0.867652,1.012367,0.487968,0.789043,0.878732,0.937757,0.843104,0.807609,1.006055,0.893318,0.926600,1.0
3,0.877682,0.757181,0.199315,0.403387,0.787560,0.876528,0.906068,0.518300,0.779388,0.775062,0.626086,0.045289,0.549311,0.749273,0.462361,0.855350,0.635400,0.926877,0.892103,0.651145,0.803821,0.816685,0.808445,0.917898,0.091059,0.511825,0.803781,0.875174,0.411465,0.975981,0.874442,0.692119,0.908366,0.889825,0.970523,0.952618,0.946301,0.0
4,0.791914,0.484325,0.087148,0.220791,0.559271,0.826807,0.898394,0.468803,0.596773,0.830030,0.616970,0.144546,0.503704,0.728342,0.439682,0.719154,0.604525,0.914425,0.938034,0.716508,0.787881,0.842529,0.672677,0.875571,0.354072,0.899181,0.894967,0.855379,0.408157,0.809717,0.956932,0.880403,0.807772,0.875958,0.974903,0.893201,0.941135,1.0
5,0.597969,0.599293,0.194503,0.376168,0.676562,0.812831,0.932033,0.481125,0.739435,0.936862,0.683657,1.089506,0.735888,0.935072,0.595323,0.652680,0.829114,0.986247,1.000920,0.705240,0.967625,0.874115,0.689969,0.853383,0.211328,0.572178,0.923301,0.961063,0.442884,0.928934,0.756959,0.799572,1.015357,0.588474,0.985477,0.914833,0.959898,1.0
6,0.718731,0.708797,0.136669,0.534425,0.939280,0.801110,0.894835,0.502526,0.843639,0.911822,0.675149,0.568458,0.576656,0.832304,0.486100,0.759921,0.573661,0.915071,0.916707,0.824139,0.968082,0.888859,0.481036,0.892760,0.127279,0.461535,0.846715,0.887935,0.741900,0.828294,0.882336,0.819573,0.919022,0.894816,0.982890,0.892490,0.919714,1.0
7,0.804704,0.491677,0.092098,0.371994,0.706488,0.817142,0.938591,0.478257,0.598019,0.916792,0.615873,0.286794,0.891385,0.888535,0.632444,0.759225,0.597665,0.895206,0.907384,0.840683,0.748924,0.876740,0.734584,0.932614,0.373547,0.883864,0.863955,0.824959,0.184651,0.833646,0.770197,0.874692,0.920535,0.788736,0.976807,0.907792,0.927982,0.0
8,0.730172,0.836495,0.330310,0.585195,0.713990,0.845455,0.868941,0.525415,0.722837,0.654491,0.653109,0.572342,0.340270,0.873584,0.704883,0.781984,0.701324,0.878592,0.844446,0.930464,0.929907,0.902509,0.689378,0.891508,0.795883,0.848949,0.889051,0.902559,0.545239,0.815716,0.902699,0.827845,0.842108,0.720832,0.972796,0.883924,0.916053,1.0
9,0.899835,0.745887,0.428086,0.425016,0.786682,0.842514,0.912202,0.530998,0.845691,0.829440,0.676038,0.130090,0.524154,0.854970,0.581403,0.785462,0.602414,0.974239,0.946574,0.705961,0.813439,0.858630,0.689655,0.903795,0.575657,1.033937,0.825560,0.878946,0.934325,0.833019,0.944482,0.863729,0.870698,0.820564,0.983408,0.974774,0.996887,0.0


In [11]:
# save to csv file

# 
Xy_NCa_PCa_md.to_csv('data/Xy_test_NCa_PCa_md.csv', index=False)